In [1]:
# CYTools v1.2.6

import copy
import cytools
from cytools import Polytope, Cone

# This function returns the Esd_3 of the standard 3-dimensional 
# simplex in R^4, dilated by a factor of 3. The simplex is 
# conv{3e_1,...,3e_4}, where e_1,...,e_4 is the standard basis 
# of R^4. The function returns a list of simplices, where each 
# simplex is a list of points, and each point is a list of 
# non-negative integers. Thus, the function returns a list of 
# lists of lists of integers.
def get_3_subdivision_of_3_simplex():
    triangles = []
    for i in range(3):
        for j in range(3):
            for k in range(3):
                color_scheme = []
                for t in range(12):
                    r = (t > 4*i) + (t > 1 + 4*j) + (t > 2 + 4*k)
                    color_scheme.append(r)
                triangle = []
                for j1 in range(4):
                    p = [0, 0, 0, 0]
                    for i1 in range(3):
                        p[color_scheme[j1 + i1*4]] += 1
                    triangle.append(p)
                triangles.append(triangle)
    return triangles

# This function returns the Esd_3 of the standard 4-dimensional 
# simplex in R^5, dilated by a factor of 3. The simplex is 
# conv{3e_1,...,3e_5}, where e_1,...,e_5 is the standard basis 
# of R^5. The function returns a list of simplices, where each 
# simplex is a list of points, and each point is a list of 
# non-negative integers. Thus, the function returns a list of 
# lists of lists of integers.
def get_3_subdivision_of_4_simplex():
    triangles = []
    for i1 in range(3):
        for i2 in range(3):
            for i3 in range(3):
                for i4 in range(3):
                    color_scheme = []
                    for t in range(15):
                        r = (t > 5*i1) + (t > 1 + 5*i2) + (t > 2 + 5*i3) + \
                            (t > 3 + 5*i4)
                        color_scheme.append(r)
                    triangle = []
                    for j in range(5):
                        p = [0, 0, 0, 0, 0]
                        for i in range(3):
                            p[color_scheme[j + i*5]] += 1
                        triangle.append(p)
                    triangles.append(triangle)
    return triangles

# Given sigma in Esd_4(P(U_i)), this function returns the 
# subdivision of prism(sigma, sigma+w), where w = (1,1,1,-1,-1,-1)^T. 
# The function returns a list of simplices, where each simplex 
# is a list of points, and each point is a list of non-negative 
# integers. Thus, the function returns a list of lists of 
# lists of integers.
def get_prism_subdivision(sigma):
    delta = copy.deepcopy(sigma)
    for point in delta:
        point[0] += 1
        point[1] += 1
        point[2] += 1
        point[3] -= 1
        point[4] -= 1
        point[5] -= 1
    triangles = []
    for i in range(4):
        triangle = []
        for t in range(4):
            if (t <= i):
                triangle.append(sigma[t])
            if (t >= i):
                triangle.append(delta[t])
        triangles.append(triangle)
    return triangles

# This function returns a list of maximal cones of Pi. Each 
# maximal cone is a list of ray generators, and each ray 
# generator is a list of integers. Thus, the function returns 
# a list of lists of lists of integers.
def get_maximal_cones_of_Pi():
    cones = []
    for i in range(3):
        triangles = get_3_subdivision_of_4_simplex()
        for triangle in triangles:
            for point in triangle:
                point.insert(i, 0)
                point[0] -= 1
                point[1] -= 1
                point[2] -= 1
        cones += triangles
    for i in range(3, 6):
        triangles = get_3_subdivision_of_4_simplex()
        for triangle in triangles:
            for point in triangle:
                point.insert(i, 0)
                point[3] -= 1
                point[4] -= 1
                point[5] -= 1
        cones += triangles
    for i in range(3):
        for j in range(3, 6):
            triangles = get_3_subdivision_of_3_simplex()
            for triangle in triangles:
                for point in triangle:
                    point.insert(i, 0)
                    point.insert(j, 0)
                    point[0] -= 1
                    point[1] -= 1
                    point[2] -= 1
                cones += get_prism_subdivision(triangle)
    return cones

# This is the polytope P, the same P mentioned in the paper. 
# It has 12 vertices and 111 integer points, including the origin (0).
P = Polytope([[3, 0, 0, -1, -1],
              [0, 3, 0, -1, -1],
              [0, 0, 3, -1, -1],
              [0, 0, 0, 2, -1],
              [0, 0, 0, -1, 2],
              [0, 0, 0, -1, -1],
              [2, -1, -1, 0, 0],
              [-1, 2, -1, 0, 0],
              [-1, -1, 2, 0, 0],
              [-1, -1, -1, 3, 0],
              [-1, -1, -1, 0, 3],
              [-1, -1, -1, 0, 0]]);

# This small patch of code converts the data into a format 
# digestible by CYTools. It uses an isomorphism M -> Z^5 that
# removes the last coordinate. Additionally, it adds the point 0 
# to each cone to convert a fan into a star triangulation. 
# In CYTools, each simplex is represented as a set of indices 
# of points, not the points themselves. This is why the 
# "rays.index(...)" part is necessary.
points = P.points()
number_of_points_in_P = len(points)
cones_nonreduced = get_maximal_cones_of_Pi()
rays = points.tolist()
triangles = []
for cone in cones_nonreduced:
    converted_cone = []
    for ray in cone:
        ray = ray[:-1]
        converted_cone.append(rays.index(ray))
    converted_cone.append(rays.index([0, 0, 0, 0, 0]))
    triangles.append(converted_cone)
    
# The flag "check_input_simplices=True" is used to ensure that
# the triangulation is well-defined. It is not necessary,
# since we already kn ow this from Theorem 4.2, but it doesn't
# hurt to double-check!
T = cytools.triangulation.Triangulation(pts=range(number_of_points_in_P),
                                        poly=P,
                                        simplices=triangles,
                                        check_input_simplices=True);

# This function returns "True" if the triangulation T is 
# projective (in the package, it is called "regular," which is 
# also a common name for such triangulations).
# It writes "True" in the output in a very short time!
print(T.is_regular())

# Also writes "True", which is expected.
print(T.is_star())


True
True
